# Introduction
This IPython notebook explains a basic workflow two tables using py_entitymatching. The goal is to come up with a workflow to match books from Goodreads and Amazon. Specifically, we want to maximize F1. The datasets contain information about the books.

First, we need to import py_entitymatching package and other libraries as follows:

In [1]:
import py_entitymatching as em
import pandas as pd
import os
import sys
from timeit import default_timer as timer

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.5.2 (default, Sep 14 2017, 22:51:06) 
[GCC 5.4.0 20160609]
pandas version: 0.20.3
magellan version: 0.3.0


Matching two tables typically consists of the following three steps:

1. Reading the input tables

2. Blocking the input tables to get a candidate set

3. Matching the tuple pairs in the candidate set

## Read input tables

In [3]:
source1 = 'source1_cleaned.csv'
source2 = 'source2_cleaned.csv'

# Read the data
data_source1 = em.read_csv_metadata(source1)
data_source2 = em.read_csv_metadata(source2)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
# Set the metadata
em.set_key(data_source1, 'ID')
em.set_key(data_source2, 'ID')

True

In [5]:
print('Number of tuples in A: ' + str(len(data_source1)))
print('Number of tuples in B: ' + str(len(data_source2)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(data_source1)*len(data_source2)))

Number of tuples in A: 3387
Number of tuples in B: 3001
Number of tuples in A X B (i.e the cartesian product): 10164387


In [6]:
data_source1.head(2)

,ID,Name,Author,Publisher,Publishing_Date,Format,Pages,Rating
0,0,Age of Myth: Book One of The Legends of the First Empire,Michael J. Sullivan,Del Rey,2017-1-31,Paperback,464.0,4.5
1,1,Rise of the Dragons (Kings and Sorcerers--Book 1),Morgan Rice,Morgan Rice,2017-8-4,Hardcover,217.0,4.1


In [7]:
data_source2.head(2)

,ID,Name,Author,Publisher,Publishing_Date,Format,Pages,Rating
0,0,Brides of Fantasy,Vanilla Orchid Books,NaN,NaN,Kindle Edition,NaN,0.00
1,1,The Italian Secretary: A Further Adventure Of Sherlock Holmes,Caleb Carr,Sphere,2015-11-27,Paperback,288.0,3.19


In [8]:
# Display the keys of the input tables
em.get_key(data_source1), em.get_key(data_source2)

('ID', 'ID')

Here we will proceed without downsampling the datasets and use the entire dataset. 

## Block tables to get candidate set
Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.

Here we use overlap blocker on the name of the book to and we set the number of overlapping words to be 3, to consider the pair a match.

In [9]:
start = timer()

# Overlap blocker
overlapBlocker = em.OverlapBlocker()

C1 = overlapBlocker.block_tables(data_source1, data_source2, 'Name', 'Name', word_level=True, overlap_size=3, 
                    l_output_attrs=['Name', 'Author', 'Publisher'], 
                    r_output_attrs=['Name', 'Author', 'Publisher'],
                    show_progress=False)

end = timer()
print(end - start)

1.4566700000000026


In [10]:
len(C1)

118876

In [11]:
C1.head()

,_id,ltable_ID,rtable_ID,ltable_Name,ltable_Author,ltable_Publisher,rtable_Name,rtable_Author,rtable_Publisher
0,0,3,1,A Quest of Heroes: Book #1 in the Sorcerer's Ring,Morgan Rice,Morgan Rice,The Italian Secretary: A Further Adventure Of Sherlock Holmes,Caleb Carr,Sphere
1,1,27,1,A Child's Work: The Importance of Fantasy Play,Vivian Gussin Paley,University of Chicago Press,The Italian Secretary: A Further Adventure Of Sherlock Holmes,Caleb Carr,Sphere
2,2,37,1,The Ballad of Titha Mae: A Fantasy in Twelve Parts,Jon B. Dalvy,Independently published,The Italian Secretary: A Further Adventure Of Sherlock Holmes,Caleb Carr,Sphere
3,3,52,1,Make A Witch (A Wicked Witches of the Midwest Fantasy) (Volume 3),Amanda M. Lee,CreateSpace Independent Publishing Platform,The Italian Secretary: A Further Adventure Of Sherlock Holmes,Caleb Carr,Sphere
4,4,70,1,"Fantasy Coloring Adventure: A Magical World of Fantasy Creatures, Enchanted Animals, and Whimsic...",Jade Summer,CreateSpace Independent Publishing Platform,The Italian Secretary: A Further Adventure Of Sherlock Holmes,Caleb Carr,Sphere


## Debug blocker output
The number of tuple pairs considered for matching is reduced to 118876 (from 10164387), but we would want to make sure that the blocker did not drop any potential matches.

In [12]:
# Debug blocker output
dbg = em.debug_blocker(C1, data_source1, data_source2, output_size=200)

In [13]:
dbg.head()

,_id,ltable_ID,rtable_ID,ltable_Name,ltable_Author,ltable_Publisher,ltable_Publishing_Date,ltable_Format,rtable_Name,rtable_Author,rtable_Publisher,rtable_Publishing_Date,rtable_Format
0,0,636,1083,The Year's Best Dark Fantasy & Horror 2015 Edition,Paula Guran,Prime Books,2015-6-23,Paperback,Rock On: The Greatest Hits of Science Fiction & Fantasy,Paula Guran,Prime Books,2012-09-25,Paperback
1,1,2798,728,Tarzan of the Apes,Edgar Rice Burroughs,Digireads.com,2016-1-4,Kindle,The Resurrection of Jimber Jaw (Fantasy & Horror Classics),Edgar Rice Burroughs,NaN,NaN,Kindle Edition
2,2,2798,1549,Tarzan of the Apes,Edgar Rice Burroughs,Digireads.com,2016-1-4,Kindle,At the Earth's Core/Pellucidar,Edgar Rice Burroughs,NaN,NaN,Kindle Edition
3,3,447,1083,The Year’s Best Dark Fantasy & Horror 2016 Edition,Paula Guran,Prime Books,2016-7-19,Paperback,Rock On: The Greatest Hits of Science Fiction & Fantasy,Paula Guran,Prime Books,2012-09-25,Paperback
4,4,1059,1197,Sweet Life 2: Erotic Fantasies for Couples,Violet Blue,Cleis Press,NaN,Paperback,Lips Like Sugar: Women's Erotic Fantasies,Violet Blue,Cleis Press,2006-02-24,Paperback


As we see here, an overlap factor of 3 on the name of the book leads to a lot of false matches. This is because book names contains a lot of stop words like "The" and "Of. Hence let's try attribute equivalenec matching on the author of the book. 

We will pass the candidate set obtained in the first step for attribute equivalence matching.

In [14]:
start = timer()

# Attribute equivalence blocker
ab = em.AttrEquivalenceBlocker()
C2 = ab.block_candset(C1, l_block_attr='Author', r_block_attr='Author', show_progress=False)

end = timer()
print(end - start)

0.2507619999999946


In [15]:
len(C2)

333

In [16]:
# Display first two rows from C2
C2.head(5)

,_id,ltable_ID,rtable_ID,ltable_Name,ltable_Author,ltable_Publisher,rtable_Name,rtable_Author,rtable_Publisher
1106,1106,1281,22,"The Magazine of Fantasy and Science Fiction, July 1969 (Volume 37, No. 1)",Fritz Leiber,Mercury Press,"Gather, Darkness! (Nucleus Fantasy & Science Fiction)",Fritz Leiber,Collier Books
1279,1279,31,26,Final Fantasy V (Boss Fight Books),Chris Kohler,Boss Fight Books,Final Fantasy V,Chris Kohler,Boss Fight Books
3232,3232,620,94,Draw More Furries: How to Create Anthropomorphic Fantasy Creatures,Jared Hodges,IMPACT Books,Draw More Furries: How to Create Anthropomorphic Fantasy Animals,Jared Hodges,Impact
3387,3387,529,97,Dismantling the Fantasy: An Invitation to the Fullness of Life,Darryl Bailey,Non-Duality,Dismantling the Fantasy: An Invitation to the Fullness of Life,Darryl Bailey,Non-Duality
3701,3701,1165,104,Fantasy Art Expedition,Finlay Cowan,IMPACT Books,Fantasy Art Expedition,Finlay Cowan,Impact


We can see here that we already have some matches. Since the number of matches has dropped to just 333 from 10164387, we decided to stop debugging the blocking step and proceed with training a matcher.

In [20]:
# Saving the tuples which survived the blocking step
C2.to_csv("TuplesAfterBlocking.csv", encoding='utf-8', index=False)

## Labeling the candidate set
We labeled the tuples from the previous step as a match or not. 1 indicates a match and 0 indicates a non match. We did not use the <i>label_table</i> function.

Labeling 333 tuples took roughly 20 minutes.

In [24]:
# Load the labeled set
S = em.read_csv_metadata('GoldenData.csv', key='_id', ltable=data_source1, rtable=data_source2, 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID')

In [25]:
len(S)

333

## Splitting the labeled data into development and evaluation set
In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [26]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(S, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [27]:
len(I), len(J)

(233, 100)

### Save Set I and Set J

In [33]:
I.to_csv("SetI.csv", encoding='utf-8', index=False)
J.to_csv("SetJ.csv", encoding='utf-8', index=False)

## Selecting the best learning-based matcher
Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [57]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features
Here we use the automatically generated features

In [82]:
# Generate features
feature_table = em.get_features_for_matching(data_source1, data_source2, validate_inferred_attr_types=False)

In [83]:
# List the names of the features generated
feature_table['feature_name']

0                                           ID_ID_exm
1                                           ID_ID_anm
2                                      ID_ID_lev_dist
3                                       ID_ID_lev_sim
4                           Name_Name_jac_qgm_3_qgm_3
5                       Name_Name_cos_dlm_dc0_dlm_dc0
6                                       Name_Name_mel
7                                  Name_Name_lev_dist
8                                   Name_Name_lev_sim
9                       Author_Author_jac_qgm_3_qgm_3
10                  Author_Author_cos_dlm_dc0_dlm_dc0
11                  Author_Author_jac_dlm_dc0_dlm_dc0
12                                  Author_Author_mel
13                             Author_Author_lev_dist
14                              Author_Author_lev_sim
15                                  Author_Author_nmw
16                                   Author_Author_sw
17                Publisher_Publisher_jac_qgm_3_qgm_3
18            Publisher_Publ

We remove the **Rating** column, as the books are rated differently on Amazon and Goodreads. Hence we will drop those features here as well.

In [84]:
# Drop publishing date, rating related features
feature_table = feature_table.drop([0, 1, 2, 3, 35, 36, 37 ,38])

### Converting the development set to feature vectors

In [85]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='match',
                            show_progress=False)

In [86]:
# Display first few rows
H.head(5)

,_id,ltable_ID,rtable_ID,Name_Name_jac_qgm_3_qgm_3,Name_Name_cos_dlm_dc0_dlm_dc0,Name_Name_mel,Name_Name_lev_dist,Name_Name_lev_sim,Author_Author_jac_qgm_3_qgm_3,Author_Author_cos_dlm_dc0_dlm_dc0,...,Publishing_Date_Publishing_Date_lev_sim,Publishing_Date_Publishing_Date_jar,Publishing_Date_Publishing_Date_jwn,Publishing_Date_Publishing_Date_exm,Publishing_Date_Publishing_Date_jac_qgm_3_qgm_3,Pages_Pages_exm,Pages_Pages_anm,Pages_Pages_lev_dist,Pages_Pages_lev_sim,match
159,72630,663,1866,0.660000,0.771517,0.936852,11.0,0.750000,1.0,1.0,...,0.6,0.810714,0.867500,0.0,0.222222,0.0,0.942149,2.0,0.6,0
150,67280,547,1743,1.000000,1.000000,1.000000,0.0,1.000000,1.0,1.0,...,0.5,0.672222,0.737778,0.0,0.222222,NaN,NaN,NaN,NaN,1
106,47902,2553,1218,0.044944,0.066227,0.618121,97.0,0.248062,1.0,1.0,...,0.5,0.685185,0.779630,0.0,0.157895,0.0,0.572890,3.0,0.4,0
214,95446,663,2418,0.692308,0.771517,0.913183,12.0,0.755102,1.0,1.0,...,0.5,0.763095,0.810476,0.0,0.157895,0.0,0.796053,3.0,0.4,0
90,36024,727,921,0.688889,0.577350,0.908225,24.0,0.571429,1.0,1.0,...,0.6,0.677778,0.774444,0.0,0.142857,0.0,0.800000,3.0,0.4,0


#### Check for missing values

In [87]:
H.isnull().sum()

_id                                                 0
ltable_ID                                           0
rtable_ID                                           0
Name_Name_jac_qgm_3_qgm_3                           0
Name_Name_cos_dlm_dc0_dlm_dc0                       0
Name_Name_mel                                       0
Name_Name_lev_dist                                  0
Name_Name_lev_sim                                   0
Author_Author_jac_qgm_3_qgm_3                       0
Author_Author_cos_dlm_dc0_dlm_dc0                   0
Author_Author_jac_dlm_dc0_dlm_dc0                   0
Author_Author_mel                                   0
Author_Author_lev_dist                              0
Author_Author_lev_sim                               0
Author_Author_nmw                                   0
Author_Author_sw                                    0
Publisher_Publisher_jac_qgm_3_qgm_3                16
Publisher_Publisher_cos_dlm_dc0_dlm_dc0            16
Publisher_Publisher_jac_dlm_

#### Impute missing values with mean

In [88]:
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'match'],
                strategy='most_frequent')

### Selecting the best matcher using cross-validation
Now, we select the best matcher using k-fold cross-validation. We use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [89]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'match'],
        k=10,
        target_attr='match', metric_to_select_matcher='f1', random_state=42)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.765079,0.697222,0.708449
1,RF,0.900000,0.718056,0.786923
2,SVM,0.766667,0.263889,0.385238
3,LinReg,0.943333,0.672222,0.766883
4,LogReg,0.848333,0.673611,0.739520
5,NaiveBayes,0.789762,0.730556,0.751382
